# Installing Dependencies

In [17]:
using DelimitedFiles
using NearestNeighbors: BallTree, KDTree, BruteTree, knn
using LinearAlgebra: norm
using CUDA

## Using General Arrays

## Using kNN

In [97]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(kdtree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,:], 14, true)
        index = hcat(index, idxs)
    end
    return index
end

function force(X, r_max, s, K, NN)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs = NN[:,i]
        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

force (generic function with 1 method)

In [95]:
R_Agg = 16

@time X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5

size(X)

  0.048640 seconds (565.11 k allocations: 15.185 MiB, 16.11% gc time)


(6064, 3)

In [101]:
@time NN = NearNeighbor(X, t, t_knn)

  0.308907 seconds (36.27 k allocations: 1.922 GiB, 24.82% gc time)


14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   40   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   41   41   42   43     6023  6024  6025  6030  6031  6033
  32   34   34    9    9   10   11     6055  6057  6057  6031  6032  6032
   2    3    2    8   10   11    6     6056  6056  6060  6063  6064  6063
   4    1    6    5    4    5   35  …  6058  6061  6033  6059  6062  6061
   5    6    7   32    6    7    3     6060  6059  6064  6058  6060  6060
  24    5   26    1   33   34   51     6031  6032  6017  6037  6059  6039
  26   25   28   48    2    3   34     6062  6063  6034  6039  6038  6041
  41   27   43   33    1    2   36     6063  6064  6032  6023  6040  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   20   21   47   32   33   50     6032  6031  6016  6044  6045  6046
  42   21   44 

In [117]:
@time X = X |> cu
@time NN = NN |> cu

  0.000007 seconds
  0.000007 seconds


14×6064 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   40   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   41   41   42   43     6023  6024  6025  6030  6031  6033
  32   34   34    9    9   10   11     6055  6057  6057  6031  6032  6032
   2    3    2    8   10   11    6     6056  6056  6060  6063  6064  6063
   4    1    6    5    4    5   35  …  6058  6061  6033  6059  6062  6061
   5    6    7   32    6    7    3     6060  6059  6064  6058  6060  6060
  24    5   26    1   33   34   51     6031  6032  6017  6037  6059  6039
  26   25   28   48    2    3   34     6062  6063  6034  6039  6038  6041
  41   27   43   33    1    2   36     6063  6064  6032  6023  6040  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   20   21   47   32   33   50     6032  6031  6016  6044 

In [136]:
X[NN,:]

14×6064×3 CuArray{Float32, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 -17.98  -15.98  -13.98  -18.98  -16.98  …  17.98  19.98  14.98  16.98  18.98
 -17.98  -15.98  -13.98  -19.98  -15.98     18.98  20.98  14.98  16.98  18.98
 -16.98  -16.98  -12.98  -17.98  -17.98     16.98  18.98  13.98  15.98  19.98
 -18.98  -14.98  -14.98  -17.98  -17.98     18.98  18.98  15.98  17.98  17.98
 -15.98  -13.98  -15.98  -19.98  -15.98     16.98  17.98  16.98  18.98  16.98
 -18.98  -17.98  -14.98  -16.98  -18.98  …  19.98  19.98  15.98  14.98  19.98
 -16.98  -14.98  -12.98  -18.98  -14.98     15.98  18.98  13.98  17.98  17.98
 -19.98  -16.98  -15.98  -17.98  -16.98     17.98  19.98  12.98  15.98  16.98
 -15.98  -17.98  -11.98  -18.98  -15.98     16.98  21.98  16.98  14.98  20.98
 -17.98  -13.98  -13.98  -16.98  -17.98     18.98  17.98  14.98  18.98  18.98
 -17.98  -15.98  -13.98  -20.98  -16.98  …  17.98  19.98  14.98  16.98  18.98
 -16.98  -15.98  -12.98  -18.98  -14.98     19.98  21.98  15.98  16.98  19.9

In [123]:
@time X_Cu = X[NN,:]

  0.000221 seconds (18 allocations: 832 bytes)


14×6064×3 CuArray{Float32, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 -17.98  -15.98  -13.98  -18.98  -16.98  …  17.98  19.98  14.98  16.98  18.98
 -17.98  -15.98  -13.98  -19.98  -15.98     18.98  20.98  14.98  16.98  18.98
 -16.98  -16.98  -12.98  -17.98  -17.98     16.98  18.98  13.98  15.98  19.98
 -18.98  -14.98  -14.98  -17.98  -17.98     18.98  18.98  15.98  17.98  17.98
 -15.98  -13.98  -15.98  -19.98  -15.98     16.98  17.98  16.98  18.98  16.98
 -18.98  -17.98  -14.98  -16.98  -18.98  …  19.98  19.98  15.98  14.98  19.98
 -16.98  -14.98  -12.98  -18.98  -14.98     15.98  18.98  13.98  17.98  17.98
 -19.98  -16.98  -15.98  -17.98  -16.98     17.98  19.98  12.98  15.98  16.98
 -15.98  -17.98  -11.98  -18.98  -15.98     16.98  21.98  16.98  14.98  20.98
 -17.98  -13.98  -13.98  -16.98  -17.98     18.98  17.98  14.98  18.98  18.98
 -17.98  -15.98  -13.98  -20.98  -16.98  …  17.98  19.98  14.98  16.98  18.98
 -16.98  -15.98  -12.98  -18.98  -14.98     19.98  21.98  15.98  16.98  19.9